In [1]:
# Sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder

# DiCE imports
import dice_ml
from dice_ml.utils import helpers  # helper functions

In [2]:
import pandas as pd
import numpy as np

import os
import pickle
import time
import random
from tqdm.notebook import tqdm
from scipy.spatial.distance import cdist

In [3]:
#from MultiClass_CP_ILS import MultiClass_CP_ILS

In [3]:
DS_PATH="./dataset/MultiClass/"

In [4]:
# train_set=pd.read_csv(DS_PATH+"/BodyPerformance/bodyPerformance_Train2.csv")
# test_set=pd.read_csv(DS_PATH+"/BodyPerformance/bodyPerformance_Test2.csv")


# train_set=pd.read_csv(DS_PATH+"Thyroid/thyroid_train2.csv")
# test_set=pd.read_csv(DS_PATH+"Thyroid/thyroid_test2.csv")

# train_set=pd.read_csv(DS_PATH+"Wine/wine_train2.csv")
# test_set=pd.read_csv(DS_PATH+"Wine/wine_test2.csv")

train_set=pd.read_csv(DS_PATH+"/K_EMOCON/SUB_5/sub_5_train2.csv")
test_set=pd.read_csv(DS_PATH+"/K_EMOCON/SUB_5/sub_5_test2.csv")


train_set.iloc[:,:-1]=train_set.iloc[:,:-1].astype(np.float64)
test_set.iloc[:,:-1]=test_set.iloc[:,:-1].astype(np.float64)

X_train=train_set.iloc[:,:-1].astype(np.float64)
X_test=test_set.iloc[:,:-1].astype(np.float64)
y_train=train_set.iloc[:,-1]
y_test=test_set.iloc[:,-1]

In [5]:
from copy import deepcopy

from xgboost import XGBClassifier
bbox_chosen = XGBClassifier(n_estimators=60, reg_lambda=3, eval_metric='logloss', random_state=42)
bbox = XGBClassifier(n_estimators=60, reg_lambda=3, eval_metric='logloss', random_state=42)
bbox.fit(X_train, y_train)


# from sklearn.ensemble import RandomForestClassifier
# bbox_chosen=RandomForestClassifier(n_estimators=100, random_state=42)
# bbox = deepcopy(bbox_chosen)
# bbox.fit(X_train, y_train)

# from sklearn.neural_network import MLPClassifier

# bbox_chosen=MLPClassifier(random_state=42, max_iter=200)
# bbox = deepcopy(bbox_chosen)
# bbox.fit(X_train, y_train)

# from sklearn.svm import SVC
# bbox_chosen = SVC(kernel="rbf", probability=True, random_state=42)
# bbox = deepcopy(bbox_chosen)
# bbox.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=60,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [6]:
#Fixed: doors, persons, lug_boot → These are part of the car's design.

In [6]:
numeric_features=train_set.columns.difference(["class"]).values
#numeric_features=[]
#numeric_features=["x","y","z","E4-BVP","E4-HR","E4-IBI","E4-TEMP","Attention","delta","lowAlpha","highAlpha","lowBeta","highBeta","lowGamma","middleGamma","theta","Meditation"]

In [7]:
#categorical_features=['price','maint','doors','persons','lug_boot','safety']

In [7]:
# features_to_vary=["body fat_%", "diastolic","systolic","gripForce","sit and bend forward_cm","sit-ups counts","broad jump_cm"]
#features_to_vary=["price","maint","safety"]
#features_to_vary=["attr1","attr2","attr3","attr4","attr5","attr6","attr7","attr8","attr9","attr10","attr11","attr12","attr13","attr14","attr15","attr16","attr17","attr18","attr19","attr20","attr21"]
#features_to_vary=["volatile acidity","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","sulphates"]
features_to_vary=["E4-BVP","E4-HR","E4-IBI","E4-TEMP","Attention","delta","lowAlpha","highAlpha","lowBeta","highBeta","lowGamma","middleGamma","theta","Meditation"]

#numeric_features=["x","y","z","E4-BVP","E4-HR","E4-IBI","E4-TEMP","Attention","delta","lowAlpha","highAlpha","lowBeta","highBeta","lowGamma","middleGamma","theta","Meditation"]

In [10]:
#numeric_features=["attr1","attr17","attr18","attr19","attr20","attr21"]
#categorical_feature=["attr2","attr3","attr3","attr4","attr5","attr6","attr7","attr8","attr9","attr10","attr11","attr12","attr13","attr14","attr15","attr16"]

In [8]:
# Step 1: dice_ml.Data
d = dice_ml.Data(dataframe=train_set, continuous_features=list(numeric_features), outcome_name='class')
#d = dice_ml.Data(dataframe=train_set, continuous_features=[], categorical_features=categorical_features, outcome_name='class')

# Using sklearn backend
m = dice_ml.Model(model=bbox, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="genetic")

In [19]:
e1 = exp.generate_counterfactuals(sampled_test_set.iloc[0:1,:-1], total_CFs=5, desired_class=3,features_to_vary=features_to_vary,verbose=False )
e1.visualize_as_dataframe(show_only_changes=False)
X_cfs = e1.cf_examples_list[0].final_cfs_df
#X_cfs

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]


UserConfigValidationException: Desired class not present in training data!

In [31]:
#ens_latent_spaces=MultiClass_CP_ILS(bbox,train_set,test_set,bbox_chosen,"class",mode="1v1")

In [32]:
#ens_latent_spaces.cdist_sample_distance(X_test[0:1],X_cfs.iloc[:,:-1])

In [9]:
flatten = lambda m: [item for row in m for item in row]
#Fix as starting column_name the name of the class that for sure it is not present
column_name=X_test.columns[-1]
idx_num_cat=[]

#Detection of one hot categorical feature, that contains _ and consider as unique variable
for index, column in enumerate(X_test.columns[:]):

    if (not column.startswith(column_name)):
        new_list=[]
        idx_num_cat.append(new_list)
        column_name=column.split("_")[0]
    
    new_list.append(index)
    
idx_num_cat=idx_num_cat

#Save separately the numerical and categorical indexes 
idx_num = flatten([l for l in idx_num_cat if len(l)==1])
idx_cat = flatten([l for l in idx_num_cat if len(l)>1])

In [10]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(test_set.iloc[:,:-1], test_set.iloc[:,-1], test_size=100, stratify=test_set.iloc[:,-1], random_state=42)
sampled_test_set=pd.concat([X_test_2,y_test_2],axis=1)
sampled_test_set

,x,y,z,E4-BVP,E4-EDA,E4-HR,E4-IBI,E4-TEMP,Attention,delta,lowAlpha,highAlpha,lowBeta,highBeta,lowGamma,middleGamma,theta,Meditation,class
37,0.483334,0.372881,0.617143,0.568470,0.086744,0.232859,0.957471,0.274865,0.547799,0.040496,0.150645,0.023792,0.038266,0.018103,0.020138,0.154611,0.034541,0.656950,3
3090,0.697617,0.389831,0.317142,0.609960,0.071485,0.218847,0.040816,0.053763,0.413694,0.065485,0.074065,0.030883,0.026265,0.012855,0.046560,0.206293,0.042535,0.592530,0
3544,0.690476,0.389831,0.219999,0.552380,0.634184,0.293864,0.964889,0.870968,0.000000,0.002533,0.003510,0.000586,0.001449,0.000236,0.000413,0.003303,0.001767,0.000000,2
3842,0.614286,0.364407,0.511429,0.564137,0.152348,0.284158,0.956937,0.440860,0.332374,0.039506,0.116025,0.036536,0.043538,0.013032,0.030339,0.112158,0.046511,0.854214,1
4485,0.485714,0.669492,0.394286,0.562174,0.090356,0.277625,0.967757,0.202957,0.483057,0.086914,0.169668,0.093611,0.074069,0.029886,0.120308,0.239276,0.118831,0.660829,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,0.433333,0.779661,0.508571,0.574988,0.703695,0.111602,0.958563,0.967742,0.000000,0.006193,0.008581,0.001432,0.003542,0.000578,0.001011,0.008076,0.004319,0.000000,1
1334,0.452381,0.389831,0.622857,0.565993,0.082641,0.157136,0.888852,0.064516,0.377521,0.045613,0.089054,0.059023,0.085176,0.038834,0.087478,0.209639,0.076109,0.460165,1
51,0.571429,0.364407,0.560000,0.568120,0.077038,0.260057,0.965794,0.204301,0.428835,0.027188,0.005760,0.042622,0.036688,0.027955,0.059270,0.613386,0.084079,0.295437,1
4091,0.616666,0.360168,0.508571,0.477531,0.197457,0.074456,0.971808,0.582661,0.146990,0.036821,0.038495,0.018469,0.022169,0.017384,0.026448,0.045011,0.031196,0.352743,2


In [11]:
def cdist_sample_distance(XA:pd.DataFrame, XB:pd.DataFrame, idx_cat,idx_num,metric=('euclidean', 'jaccard'), w=None):
        """
            The distance function used to measure the distance among to 2 distinct samples; the same used in original paper
            combining Hamming distance for categorical values and cosine distance for continous attribute.
            This method can be applied simultaneously to multiple points at time, returing a matrix of distances.

            Args:
                XA (pd.DataFrame): representing the first sample instance, can be used with multiple rows
                XB (pd.DataFrame): representing the second sample instance can be used with multiple rows
                metric (str): this is the criterion used for continuous and categorical variables
                w (Any): The weight vector for metrics that support weights (e.g., Minkowski)

            Returns:
                numpy.array: a numerical array of distances accordingly to metric between XA and XB 
        """
        metric_continuous = metric[0]
        metric_categorical = metric[1]

        if idx_cat:
            dist_categorical = cdist(XA.iloc[:, idx_cat], XB.iloc[:, idx_cat],
                                 metric=metric_categorical, w=w)
           
            ratio_categorical = len(idx_cat) / (XA.shape[1]-1)
            dist = ratio_categorical * dist_categorical

            if idx_num:
                dist_continuous = cdist(XA.iloc[:, idx_num], XB.iloc[:, idx_num],
                                    metric=metric_continuous, w=w)
                
                ratio_continuous = len(idx_num) / (XA.shape[1]-1)
                dist += ratio_continuous * dist_continuous 
        else:
            dist = cdist(XA, XB, metric=metric_continuous, w=w)

        return dist

In [12]:
#cdist_sample_distance(X_test[0:1],X_cfs.iloc[:,:-1],idx_cat,idx_num)

In [12]:
def execute_testing_on_data(test_name, test_instances, dice, mbbox,  classes, total_CFs_per_Class, features_to_vary,idx_cat,idx_num, debug=False):
    """
        This function call iteratively the methods to generate the counterfactuals and save all the results on external files
        specified by input parameters, in order to make a post-hoc analyis of the goodness of used methods.
        
    """
    # Create the directory tests if it doesn't exist
    dir_name = "tests"
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print(f"Directory '{dir_name}' created.")

    #Create FILE1 new source of test_set with instance id
    ts=test_instances.copy()
    ts=ts.iloc[:,:-1]
    ts.insert(0, 'ID_TEST', range(1, len(ts) + 1))
    ts.to_csv("./"+dir_name+"/testset_"+test_name+".csv",index=False)

    #Create FILE2 intermediary informations about test_instance and its counterfactuals
    index_position=0
    df = pd.DataFrame(columns=['ID_TEST','orig_test_class','mbbox_test_class_pred','counter_class_desired',
                               'ID_COUNTER','bbbox_counter_class_pred','mbbox_counter_class_pred','mmbox_counter_class_prob','counter_distance'])

    #Create FILE3 new counterfactuals_set with id: THIRD FILE
    id_counterfactual=1
    df_counterfactuals = pd.DataFrame()

    #SECOND COLUMN -- info related to the original class
    y_labels=test_instances.iloc[:,-1]

    #THIRD COLUMN -- info related to the prediction of mbbox of original instance
    y_mbbox_predictions=mbbox.predict(test_instances.iloc[:,:-1])


    for index in tqdm(range(0, ts.shape[0]), desc="Processing test instances"):
        
        #index used to get the exact test_instance
        test_instance=ts.iloc[index:index+1,1:]
        
        test_instance_id=ts["ID_TEST"].iloc[index]

        ris_list = []  # Store results in a list first (more efficient than repeated concat)

        for lab in classes:
            if lab != y_mbbox_predictions[index]:  # No need for list comprehension
                if (debug): 
                    print(f"Predicted: {y_mbbox_predictions[index]} and lab: {lab}")
                try:

                    # Example usage
                    e = exp.generate_counterfactuals(test_instance, total_CFs=total_CFs_per_Class, desired_class=lab, features_to_vary=features_to_vary,  verbose=debug)
                    temp=e.cf_examples_list[0].final_cfs_df
                except Exception as ex:
                    print("Exception:", ex)
                    temp=None
               
                    
                if (debug):
                    e.visualize_as_dataframe(show_only_changes=True)
                
                
                #FOURTH COLUMN: label used to get the desired class of the counterfactual
                counter_class_desired=lab
                
                if (temp is not None and not temp.empty):
                    gen_counterfactuals=temp.copy()
                    
                    #EIGHTH COLUMN: probability prediction of multi-class classifier
                    probab=mbbox.predict_proba(gen_counterfactuals.iloc[:,:-1])
                    max_prob=np.max(probab, axis=1)
                    
                    #NINTH COLUMN: distance between test sample and generated counterfactuals
                    
                    distances=cdist_sample_distance(test_instance,gen_counterfactuals.iloc[:,:-1],idx_cat,idx_num)

                    #FIVETH COLUMN: id of generated counterfactual
                    gen_counterfactuals.insert(0,'ID_COUNTER',range(id_counterfactual,id_counterfactual+gen_counterfactuals.shape[0]))

                    df_counterfactuals=pd.concat([df_counterfactuals,gen_counterfactuals.iloc[:,:-1]])
                    
                    for index_counterfactual in range(gen_counterfactuals.shape[0]):
 
                        #SIXTH COLUMN: counterfactual's class prediction of binary classifier 
                        counter_bbbox_pred=mbbox.predict(temp.iloc[index_counterfactual:index_counterfactual+1,:-1])[0]
                        #SEVENTH COLUMN: counterfactual's class prediction of multiclass classifier 
                        counter_mbbox_pred=counter_bbbox_pred
                        df.loc[index_position]={"ID_TEST":test_instance_id,
                                            "orig_test_class":y_labels.iloc[index],
                                            "mbbox_test_class_pred":y_mbbox_predictions[index],
                                            "counter_class_desired": lab,
                                            "ID_COUNTER":id_counterfactual+index_counterfactual,#gen_counterfactuals["ID_COUNTER"][index_counterfactual],
                                            "bbbox_counter_class_pred":counter_bbbox_pred,
                                            "mbbox_counter_class_pred":counter_mbbox_pred,
                                            "mmbox_counter_class_prob": float(f"{max_prob[index_counterfactual]:.4f}"),
                                            "counter_distance":float(f"{distances[0][index_counterfactual]:.4f}")
                                           }
                        index_position=index_position+1
                    id_counterfactual=id_counterfactual+gen_counterfactuals.shape[0]
                
                else:
                    df.loc[index_position]={"ID_TEST":test_instance_id,
                                            "orig_test_class":y_labels.iloc[index],
                                            "mbbox_test_class_pred":y_mbbox_predictions[index],
                                            "counter_class_desired": lab,
                                            "ID_COUNTER":None,
                                            "bbbox_counter_class_pred":None,
                                            "mbbox_counter_class_pred":None,
                                            "mmbox_counter_class_prob": None,
                                            "counter_distance":None
                                       }
                index_position=index_position+1
                        
        



    
    df_counterfactuals.to_csv("./"+dir_name+"/counterfactualset_"+test_name+".csv",index=False)
    df.to_csv("./"+dir_name+"/results_"+test_name+".csv",index=False)     

In [14]:
# class_ratios = {0: 0.33, 1: 0.33, 2: 0.34}
# test_size = 100  # Total test set size

# # Split per class
# test_samples = []

# for label, ratio in class_ratios.items():
#     class_subset = test_set[test_set['class'] == label]
#     test_count = int(test_size * ratio)  # Exact number of samples per class

#     # Stratified split for the current class
#     _, test_part = train_test_split(class_subset, test_size=test_count, random_state=42)

#     test_samples.append(test_part)


# test_set_2 = pd.concat(test_samples)

# # Shuffle to mix instances
# sampled_test_set = test_set_2.sample(frac=1, random_state=42).reset_index(drop=True)
# X_sampled=sampled_test_set.iloc[:,:-1].astype(np.float64)
# y_sampled=sampled_test_set.iloc[:,-1]

In [13]:
execute_testing_on_data("t0", sampled_test_set, exp, bbox, [0,1,2,3], 3, features_to_vary,idx_cat,idx_num, debug=False)

Processing test instances:   0%|          | 0/100 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.01it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.43it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.85it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.95it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.97it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.01it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.53it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.95it/s]

100%|██████████████████

Exception: No counterfactuals found for any of the query points! Kindly check your configuration.



100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.99it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.05it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.95it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.95it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.56it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.01it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.03it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.11s/it]

100%|██████████████████

Exception: No counterfactuals found for any of the query points! Kindly check your configuration.


...


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.72it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.69it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.72it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.69it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.02s/it]


Exception: No counterfactuals found for any of the query points! Kindly check your configuration.



100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.55it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.60it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.08it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.76s/it]


Exception: No counterfactuals found for any of the query points! Kindly check your configuration.



100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.61it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.64it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.77it/s]
